In [ ]:
import cv2
import PIL.Image as Image
import numpy as np

IMAGES_PATH = "./img/"

ROWS = 2
COLUMNS = 4
IMAGE_SAVE_PATH = './img/final.png'

height = 1280
width = 962

def biggest_contour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        # Get the area and perimeter of contour
        area = cv2.contourArea(i)
        peri = cv2.arcLength(i, True)

        # Approximate the (closed) curve and get its points
        approx = cv2.approxPolyDP(i, 0.02 * peri, True)

        # If current contour is bigger than previous one, AND it is a rectangle
        if area > max_area and len(approx) == 4:
            biggest = approx
            max_area = area

    # Squeeze to 2D matrix: (4, 1, 2) -> (4, 2)
    # biggest = np.squeeze(biggest, axis=1)

    return biggest, max_area


def reorder(points):
    """
    Reorder the points based on following notation:
    (x_min, y_min), (x_max, y_min), (x_min, y_max), (x_max, y_max)
    :param points: The 4 points to reorder
    :return: The reordered 4 points -> dimension: (4, 1, 2)
    """
    # Squeeze to 2D matrix: (4, 1, 2) -> (4, 2)
    points = np.squeeze(points, axis=1)

    points_new = np.zeros((4, 1, 2), dtype=np.int32)

    # Add the x and y values of all 4 points
    add = points.sum(1)

    # First point, with smallest x and y coordinates
    points_new[0] = points[np.argmin(add)]

    # Last point, with largest x and y coordinates
    points_new[3] = points[np.argmax(add)]

    # Subtract the x and y values of all 4 points
    diff = np.diff(points, axis=1)

    points_new[1] = points[np.argmin(diff)]
    points_new[2] = points[np.argmax(diff)]

    return points_new

In [ ]:
import cv2
import numpy as np

PATH = "04.jpg"


def doc_scan_pipeline(input=PATH, output="res.jpg"):
    img = cv2.imread(input)

    # 0. Convert given image from BGR to RGB format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    h, w, _ = img.shape
    img = cv2.resize(img, (width, height))

    # 1. Convert to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # 2. Add Gaussian blur
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 1)

    # 3. Add Canny edge detection
    img_threshold = cv2.Canny(img_blur, 100, 200, L2gradient=True)

    # 3.1 Apply dilation
    kernel = np.ones((3, 3))
    img_threshold = cv2.dilate(img_threshold, kernel, iterations=2)

    # 4. Find all the contours
    img_contours = img.copy()
    img_big_contour = img.copy()
    contours, hierarchy = cv2.findContours(img_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image=img_contours, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=5)

    # 5. Find the biggest contour
    biggest, maxArea = biggest_contour(contours)
    biggest = reorder(biggest)
    cv2.drawContours(image=img_big_contour, contours=biggest, contourIdx=-1, color=(0, 255, 0), thickness=10)

    # 5.1 Draw a rectangle, i.e., 4 lines connecting the 4 dots corresponding to the largest contour
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])

    # 6. Image Warp
    # 6.1 Calculate a 3x3 perspective transform matrix
    matrix = cv2.getPerspectiveTransform(pts1, pts2)

    # 6.2 Apply the perspective matrix to the image
    img_warp_coloured = cv2.warpPerspective(img, matrix, (width, height))

    # Save the document to disk
    cv2.imwrite(output, img_warp_coloured)

In [ ]:
doc_scan_pipeline()

In [ ]:
import cv2
import numpy as np

def stitch_images(image1, image2):
    # Загрузка изображений
    img1 = cv2.imread(image1)
    img2 = cv2.imread(image2)

    # Преобразование изображений в черно-белое
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Использование детектора ключевых точек (например, SIFT или ORB)
    detector = cv2.ORB_create()
    keypoints1, descriptors1 = detector.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = detector.detectAndCompute(gray2, None)

    # Создание объекта сопоставления
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Выполнение сопоставления ключевых точек
    matches = matcher.match(descriptors1, descriptors2)

    # Сортировка сопоставлений по расстоянию
    matches = sorted(matches, key=lambda x: x.distance)

    # Ограничение количества сопоставлений
    max_matches = 100
    matches = matches[:max_matches]

    # Извлечение координат ключевых точек из сопоставлений
    points1 = []
    points2 = []
    for match in matches:
        points1.append(keypoints1[match.queryIdx].pt)
        points2.append(keypoints2[match.trainIdx].pt)

    # Преобразование точек в массивы NumPy
    points1 = np.array(points1)
    points2 = np.array(points2)

    # Оценка гомографической матрицы
    H, _ = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Выполнение перспективного преобразования изображения
    stitched_image = cv2.warpPerspective(img1, H, (img1.shape[1] + img2.shape[1], img1.shape[0]))
    stitched_image[0:img2.shape[0], 0:img2.shape[1]] = img2

    return stitched_image

# Пример использования функции stitch_images
image1 = 'testA1.jpg'
image2 = 'testA2.jpg'
result = stitch_images(image1, image2)

# Сохранение результата
cv2.imwrite('result.jpg', result)
print("Изображения успешно склеены и сохранены в result.jpg")

Изображения успешно склеены и сохранены в result.jpg


In [ ]:
import cv2
import numpy as np

img1 = cv2.imread("res0.jpg")
img2 = cv2.imread("res1.jpg")

sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)


matcher = cv2.BFMatcher()
matches = matcher.match(des1, des2)

matches = [m for m in matches if m.distance < 100]

src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

affine_transform, _ = cv2.estimateAffine2D(src_pts, dst_pts)
img1_aligned = cv2.warpAffine(img1, affine_transform, (img1.shape[1], img1.shape[0]))

cv2.imwrite('affine.jpg', img1_aligned)


alpha = 0.5 # коэффициент смешивания
blended_image = cv2.addWeighted(img1_aligned, alpha, img2, 1-alpha, 0)

cv2.imwrite('Blended Image.jpg', blended_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2

# Открываем видеофайл
video_path = 'path/to/your/video/file.mp4'  # Укажите путь к вашему видеофайлу
cap = cv2.VideoCapture(video_path)

# Проверяем, успешно ли открыт видеофайл
if not cap.isOpened():
    print("Ошибка: не удалось открыть видеофайл")
    exit()

# Создаем пустой массив для хранения кадров
frames = []

# Счетчик кадров
frame_count = 0

# Читаем кадры из видеофайла, пока видеофайл не закончится
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Пропускаем первые 7 кадров
    if frame_count % 8 == 0:
        # Преобразуем кадр в оттенки серого, если нужно
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Добавляем кадр в массив
        frames.append(frame)

        # Отображаем кадр
        cv2.imshow("Frame", frame)

    # Прерываем цикл при нажатии клавиши "q"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1

# Закрываем окна и освобождаем ресурсы
cap.release()
cv2.destroyAllWindows()

# Выводим информацию о количестве кадров
print(f"Количество кадров: {len(frames)}")


In [ ]:
import cv2
import numpy as np

BG_COLOR = 209

def blank_image(width=1024, height=1024):
    img = np.full((height, width, 1), BG_COLOR, np.uint8)
    return img

def noisy(image):
    row, col, ch = image.shape
    mean = 0
    sigma = 10
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = gauss + image
    return noisy

if __name__ == '__main__':
    img = blank_image()
    cv2.imwrite('out.jpg', noisy(img))